In [1]:
import scipy.linalg as la
import matplotlib.pyplot as plt
import math
import numpy as np
import sys
import pathlib

/home/matyko/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [63]:
import numpy as np
import scipy.linalg

class AdaptiveLQR:
    def __init__(self, A, B, Q_base, R_base):
        """
        Initializes the Adaptive LQR Controller.
        :param A: System dynamics matrix.
        :param B: Control input matrix.
        :param Q_base: Base state cost matrix.
        :param R_base: Base control cost matrix.
        """
        self.A = A
        self.B = B
        self.Q_base = Q_base
        self.R_base = R_base

    def compute_lqr_gain(self, Q, R):
        """ Solves the Discrete Algebraic Riccati Equation for LQR gain K. """
        P = scipy.linalg.solve_discrete_are(self.A, self.B, Q, R)
        K = np.linalg.inv(self.B.T @ P @ self.B + R) @ (self.B.T @ P @ self.A)
        return K

    def adaptive_control(self, steering_error, a_y_target, velocity):
        """
        Computes the adaptive LQR control input (torque).
        :param steering_error: Steering angle error (rad).
        :param a_y_target: Target lateral acceleration (m/s²).
        :param velocity: Vehicle speed (m/s).
        :return: Torque command in range [-1, 1].
        """
        # **State Vector: [steering angle error, target lateral acceleration]**
        x = np.array([[steering_error], [a_y_target]])

        # **Adaptive Gain Scaling Based on Speed**
        # Higher speeds → More conservative control (lower Q, higher R)
        q_scale = 1 + np.tanh(np.linalg.norm(x) / (2 + velocity / 10))
        r_scale = 1 + 0.5 * (velocity / 30)  # More damping at high speeds

        Q = self.Q_base * q_scale
        R = self.R_base * r_scale

        # Compute adaptive LQR gain
        K = self.compute_lqr_gain(Q, R)

        # Compute control action: u = -Kx
        torque = float(-K @ x)

        # **Limit torque to the range [-1, 1]**
        torque = np.clip(torque, -1, 1)
        return torque

# **Example: Adaptive LQR for Steering Torque Control**
A = np.array([[1.0, 0.1], [0, 1.0]])  # State transition (simplified)
B = np.array([[0.6], [0.2]])          # Control input matrix

Q_base = np.array([[5.0, 0], [0, 1.0]])  # Base Q (penalizing steering error more)
R_base = np.array([[0.5]])               # Base R (penalizing control effort)

lqr = AdaptiveLQR(A, B, Q_base, R_base)

In [64]:

lqr.adaptive_control(0.4, -0.6, 33.0)  # Example usage

/tmp/ipykernel_2037/2427221761.py:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  torque = float(-K @ x)


-0.3846340482419734